## Import Packages

In [1]:
from tqdm import tqdm
import os

from sklearn.model_selection import train_test_split

import pandas as pd 
import random 
import numpy as np 
from torch import nn, optim 
import torch 

from torch.utils.data import Dataset, DataLoader

c:\Users\user\AppData\Local\Programs\Python\Python38\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Build Class

In [2]:
class GMF(nn.Module):
    def __init__(self, args):
        super(GMF, self).__init__()

        self.num_users = args.num_users 
        self.num_items = args.num_items 
        self.latent_dim = args.latent_dim 

        self.user_embedding = nn.Embedding(self.num_users, self.latent_dim)
        self.item_embedding = nn.Embedding(self.num_items, self.latent_dim)

        self.fc_layer = nn.Linear(args.latent_dim, 1)
        self.sigmoid = nn.Sigmoid()
        
    def forward(self, uid, iid):
        user_emb = self.user_embedding(uid)
        item_emb = self.item_embedding(iid)

        multiply_layer = torch.mul(user_emb, item_emb)
        predict = self.fc_layer(multiply_layer)
        output = self.sigmoid(predict)
        return output 

    def _init_weight(self):
        for module in self.modules():
            if isinstance(module, nn.Embedding):
                nn.init.normal_(module.weight, std=0.01)
            
            if isinstance(module, nn.Linear) and module.bias is not None:
                module.bias.zero_()

In [3]:
def set_seed(args):
    random.seed(args.seed)
    np.random.seed(args.seed)
    torch.manual_seed(args.seed)
    torch.cuda.manual_seed_all(args.seed)

In [4]:
class MFDataset(Dataset):
    def __init__(self, dataframe):
        self.user_id = dataframe.user_id
        self.item_id = dataframe.business_id
        self.labels = dataframe.stars 

    def __len__(self):
        return len(self.user_id)

    def __getitem__(self, idx):
        uid = self.user_id[idx]
        iid = self.item_id[idx]
        label = self.labels[idx]

        return (
            torch.tensor(uid, dtype=torch.long), 
            torch.tensor(iid, dtype=torch.long), 
            torch.tensor(label, dtype=torch.float)
        )


In [5]:
def sentiment_score(x):
    if x >= 3.5 : return 1
    elif x < 3.5 : return 0
    
def train(args, model, train_loader, test_loader, optimizer, criterion):
    train_losses = []
    valid_losses = []
    best_loss = float('inf')

    set_seed(args)
    model = model.to(args.device)
    model.train()
    for epoch in range(1, args.num_epochs + 1):
        train_loss, valid_loss = 0.0, 0.0

        for batch in tqdm(train_loader, desc='Training...'):
            batch = tuple(b.to(args.device) for b in batch)

            inputs = {'uid':    batch[0], 
                      'iid':    batch[1]}
            gold_y = batch[2]

            pred_y = model(**inputs).squeeze()
            loss = criterion(pred_y, gold_y)

            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

            train_loss += loss.item()

        train_loss /= len(train_loader)

        train_losses.append(train_loss)

        valid_loss = evaluate(args, model, test_loader, criterion)
        valid_losses.append(valid_loss)


        print(f'Epoch: [{epoch}/{args.num_epochs}]')
        print(f'Train Loss: {train_loss:.5f}')
        print(f'Valid Loss: {valid_loss:.5f}')

        if best_loss > valid_loss:
            best_loss = valid_loss
            if not os.path.exists(args.SAVE_PATH):
                os.makedirs(args.SAVE_PATH)
            torch.save(model.state_dict(), os.path.join(args.SAVE_PATH, f'{model._get_name()}_parameters.pt'))
    
    return {
        'train_loss': train_losses, 
        'valid_loss': valid_losses
    }


def evaluate(args, model, test_loader, criterion):
    losses = 0
    model.eval()
    with torch.no_grad():
        for batch in tqdm(test_loader, desc='Evaluating...'):
            batch = tuple(b.to(args.device) for b in batch)

            inputs = {'uid':    batch[0], 
                      'iid':    batch[1]}
            gold_y = batch[2]

            pred_y = model(**inputs).squeeze()
                
            loss = criterion(pred_y, gold_y)
        
            losses += loss.item()
        losses /= len(test_loader)
    return losses


In [6]:
class args:
    seed = 42 
    batch_size = 512
    SAVE_PATH = os.path.basename(os.getcwd())
    device = 'cuda' if torch.cuda.is_available() else 'cpu'
    lr = 1e-3

In [7]:
DATA_PATH = '../dataset/sample.csv'
d_set = pd.read_csv(DATA_PATH, encoding='utf-8-sig')

d_set = d_set.astype({'user_id':'category', 'business_id':'category'})

d_set.stars = d_set.stars.apply(sentiment_score)

In [8]:
d_train, d_test = train_test_split(d_set, train_size=0.8, random_state=args.seed)
d_valid, d_test = train_test_split(d_test, train_size=0.5, random_state=args.seed)

In [9]:
d_train.shape, d_valid.shape, d_test.shape

((80000, 4), (10000, 4), (10000, 4))

In [10]:
u_cat = d_train.user_id.cat.categories
b_cat = d_train.business_id.cat.categories 

In [11]:
d_valid.user_id = d_valid.user_id.cat.set_categories(u_cat)
d_valid.business_id = d_valid.business_id.cat.set_categories(b_cat)

d_test.user_id = d_test.user_id.cat.set_categories(u_cat)
d_test.business_id = d_test.business_id.cat.set_categories(b_cat)

In [12]:
d_train.user_id = d_train.user_id.cat.codes
d_train.business_id = d_train.business_id.cat.codes 

d_valid.user_id = d_valid.user_id.cat.codes
d_valid.business_id = d_valid.business_id.cat.codes 

d_test.user_id = d_test.user_id.cat.codes
d_test.business_id = d_test.business_id.cat.codes 

In [13]:
d_train = d_train.dropna()
d_valid = d_valid.dropna()
d_test = d_test.dropna()

d_train.reset_index(drop=True, inplace=True)
d_valid.reset_index(drop=True, inplace=True)
d_test.reset_index(drop=True, inplace=True)

In [14]:
args.num_users = d_train.user_id.nunique()
args.num_items = d_train.business_id.nunique()
args.latent_dim = 64
args.num_epochs = 10

In [15]:
model = GMF(args)

train_set = MFDataset(d_train)
valid_set = MFDataset(d_valid)
test_set = MFDataset(d_test)

train_loader = DataLoader(train_set, batch_size=args.batch_size, num_workers=4, pin_memory=True)
valid_loader = DataLoader(valid_set, batch_size=args.batch_size, num_workers=4, pin_memory=True)
test_loader = DataLoader(test_set, batch_size=args.batch_size, num_workers=4, pin_memory=True)

optimizer = optim.SGD(model.parameters(), lr=args.lr)
criterion = nn.BCELoss()

In [16]:
train_loss, valid_loss = train(args, model, train_loader, valid_loader, optimizer, criterion) 

Training...:   0%|          | 0/157 [00:00<?, ?it/s]

In [ ]:
test_loss = evaluate(args, model, test_loader, criterion)